# containers_with_enums_and_table_writer


## Create some example Containers

In [ ]:
import enum
from ctapipe.io import HDF5TableWriter
from ctapipe.core import Container, Field
from astropy import units as u
import numpy as np

In [ ]:
class WithEnum(Container):
    
    # this class could also be defined in global namespace 
    # outside this container, but this looks a bit tidier.
    # both variants work however
    class EventType(enum.Enum):
        pedestal = 1
        physics = 2
        calibration = 3
    
    event_type = Field(
        EventType.calibration, 
        f'type of event, one of: {list(EventType.__members__.keys())}'
    )

let's also make a dummy stream (generator) that will create a series of these containers

In [ ]:
def create_stream(n_event):
    data = WithEnum()
    for i in range(n_event):
        data.event_type = WithEnum.EventType(i % 3 + 1)
        yield data

In [ ]:
for data in create_stream(3):
    for key, val in data.items():
        print('{}: {}, type : {}'.format(key, val, type(val)))

## Writing the Data

In [ ]:
with HDF5TableWriter('container.h5', group_name='data') as h5_table:
    for data in create_stream(10):
        h5_table.write('table', data)

In [ ]:
!ls container.h5

## Reading the Data

In [ ]:
import pandas as pd

data = pd.read_hdf('container.h5', key='/data/table')
data.head()

#### Reading with PyTables

In [ ]:
import tables
h5  = tables.open_file('container.h5')
table = h5.root['data']['table']
table

In [ ]:
table.attrs

In [ ]:
from ctapipe.io import HDF5TableReader

def read(mode):
    
    print('reading mode {}'.format(mode))

    with HDF5TableReader('container.h5', mode=mode) as h5_table:

        for group_name in ['data/']:

            group_name = '/{}table'.format(group_name)
            print(group_name)

            for data in h5_table.read(group_name, WithEnum()):

                print(data.as_dict())

In [ ]:
read('r')